In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 10.0 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 450 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 436 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 11.0 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.0
    Uninstalling safetensors-0.4.0:
      Successfully uninstalled safetensors-0.4.0


In [2]:
# import torch
# device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
# device

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from shapely.geometry import Polygon, Point
import sys
import os
import statistics
import psutil
import copy
import pickle
import re
from abc import ABC, abstractmethod
import math
import copy
import xml.etree.ElementTree as ET
import gc
from copy import deepcopy
from pathlib import Path
from skimage import data
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import skimage
import PIL
from skimage.filters import threshold_otsu
import torchvision.models as torchmodels
from torchvision.models import convnext_tiny,ConvNeXt_Tiny_Weights

from torchvision.models import regnet_y_3_2gf,RegNet_Y_3_2GF_Weights

import torch.utils.data
import torchvision
import torch.nn as nn

import numpy as np
import torch
import torch.nn as nn
import os
import sys
import abc
from abc import ABC, abstractmethod
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score

# from generativemodels import ResidualEncoder
class FishVectEncoder(nn.Module):
    def __init__(self, size_input, num_centers,\
                        pi_of_fishvect, sigma_of_fishvect):
        '''
        Inputs.
            - size_input: the size of the input to the module, e.g., [32x100x7x7].
            - num_centers: an integer, number of visual words.
            - pi_of_fishvect: float, the pi value of the FishVect, Eq.34 of the survey.
            - sigma_of_fishvect: float, the sigma value of the FishVect, Eq.34 of the survey.
        '''
        super(FishVectEncoder, self).__init__()
        #grab the privates =====
        self.num_centers = num_centers
        self.pi_of_fishvect = pi_of_fishvect
        self.sigma_of_fishvect = sigma_of_fishvect
        #make V, the visualwords =====
        NCHW = size_input
        dim_vectors = NCHW[1]
        self.V = torch.nn.Parameter(
                      torch.randn(self.num_centers, dim_vectors).\
                      unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
                    ) #[1 x 11 x 2048 x 1 x 1]
        
    def forward(self, x):
        '''
        Inputs.
            - x: volumetric tensor of shape [20 x 2048 x 16 x 16]
        Outputs.
            - soft_assignment: [20 x 11 x 16 x 16]
            - FV_mu_part: [20 x 11*2048  x 16 x 16]
            - FV_sigma_part: #[20 x 11*2048  x 16 x 16]
        '''
        #make tao (Eq.35 of the survey) ===================
        x_resized = x.unsqueeze(1) #[N x 1 x 2048 x 16 x 16]
        x_min_V = x_resized - self.V.detach() #[N x 11 x 2048 x 16 x 16]
        tao_exponent = torch.sum(
                        -(x_min_V*x_min_V)/(2.0*self.sigma_of_fishvect*\
                                                self.sigma_of_fishvect),\
                        2
                    ) #[N x 11 x 16 x 16]
        soft_assignment = torch.nn.functional.softmax(
                                    tao_exponent,\
                                    1
                                  ) #[N x 11 x 16 x 16]
        #make FV_mu_part =======
        FV_mu_part = (x_min_V/self.sigma_of_fishvect) *\
                      soft_assignment.unsqueeze(2)*\
                      (1.0/np.sqrt(self.pi_of_fishvect))#[N x 11 x 2048 x 94 x 94]
        list_dim = list(FV_mu_part.size())
        new_list_dim = [list_dim[0]] + [-1] + list_dim[3:5]
        FV_mu_part = FV_mu_part.view(*new_list_dim) #[N x 11*2048  x 94 x 94]
        #make FV_sigma_part ======
        FV_sigma_part =\
           ((x_min_V/self.sigma_of_fishvect)*\
            (x_min_V/self.sigma_of_fishvect) - 1) *\
             soft_assignment.unsqueeze(2)*\
             (1.0/np.sqrt(2.0*self.pi_of_fishvect))#[N x 11 x 2048 x 94 x 94]
        FV_sigma_part = FV_sigma_part.view(*new_list_dim) #[N x 11*2048 x 94 x 94]
        encoded_descriptors = torch.cat([FV_mu_part, FV_sigma_part],\
                                        1)
        
        return soft_assignment, encoded_descriptors
fve = FishVectEncoder(
                         size_input=[1,504,9],
                         num_centers=5,pi_of_fishvect=0.2, sigma_of_fishvect=0.1
                      )
# [80,3,512,512])

/tmp/ipykernel_1375/1279232707.py:24: DeprecationWarning: Please use `map_coordinates` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import map_coordinates
/tmp/ipykernel_1375/1279232707.py:25: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [4]:
import pandas as pd
df=pd.read_csv("efficient.csv")
import json
df.feat = df.feat.apply(lambda x:json.loads(x))
df["feat"]

0      [[-1.1797481161434864, -0.545881941309972, -0....
1      [[0.2269251048054033, 1.1229929524691435, 0.85...
2      [[1.1918291726071466, 0.14175776077911373, 2.8...
3      [[2.3236311513910355, 1.693732421254974, 2.790...
4      [[1.162273544160464, 1.2154657209409305, 1.370...
                             ...                        
152    [[-0.6231923308293462, 0.5347237358815037, -0....
153    [[1.3145381200127018, 1.6229865244096442, 0.13...
154    [[0.8034674624606797, 0.0010932752042546179, 1...
155    [[1.3603257818013987, 0.5504098856545874, 0.55...
156    [[2.184599875414793, 0.4885522672763991, 0.609...
Name: feat, Length: 157, dtype: object

In [5]:
def filter_inner_lists(feat_list):
    return [lst for lst in feat_list if len(lst) != 8]
df['feat'] = df['feat'].apply(filter_inner_lists)
i=0
for f in df["feat"]:
    for k in f:
        if len(k)==8:
#             print(i)
            i=i+1
i

0

In [6]:
torch.tensor(df["feat"][0]).shape

torch.Size([504, 9])

In [7]:
fve(torch.tensor(df["feat"][32]))[1].squeeze(0).squeeze(1).shape

torch.Size([5040, 9])

In [8]:
df

,Unnamed: 0,feat,lab
0,0,"[[-1.1797481161434864, -0.545881941309972, -0....",1
1,1,"[[0.2269251048054033, 1.1229929524691435, 0.85...",0
2,2,"[[1.1918291726071466, 0.14175776077911373, 2.8...",0
3,3,"[[2.3236311513910355, 1.693732421254974, 2.790...",1
4,4,"[[1.162273544160464, 1.2154657209409305, 1.370...",0
...,...,...,...
152,152,"[[-0.6231923308293462, 0.5347237358815037, -0....",1
153,153,"[[1.3145381200127018, 1.6229865244096442, 0.13...",0
154,154,"[[0.8034674624606797, 0.0010932752042546179, 1...",1
155,155,"[[1.3603257818013987, 0.5504098856545874, 0.55...",1


In [9]:
# df_clus = pd.DataFrame(index=range(157), columns=['feat', 'lab'])

In [10]:
from sklearn.cluster import KMeans
import numpy as np
df_clus = pd.DataFrame(index=range(157), columns=['feat', 'lab'])
for ab in range(157):
    
    num_clusters = 10
    data=(df['feat'][ab])

    data_array = np.array(data)
    num_clusters = 10

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data_array)

    cluster_assignments = kmeans.predict(data_array)
    cluster_means = []
    for cluster_label in range(num_clusters):
        cluster_data = [data_array[i] for i in range(len(cluster_assignments)) if cluster_assignments[i] == cluster_label]
#         print(len(cluster_data),len(cluster_data[0]))
#         cluster_mean = np.mean(cluster_data, axis=0)
        fve = FishVectEncoder(
                         size_input=[1,len(cluster_data),9],
                         num_centers=7,pi_of_fishvect=0.1428571428, sigma_of_fishvect=0.1
                      )
        cluster_mean=fve(torch.tensor(cluster_data))[1].squeeze(0).squeeze(1)
        cluster_means.append(cluster_mean)
#     print(len(cluster_means),len(cluster_means[0]))
#     print(cluster_means)
    df_clus['feat'][ab]=cluster_means
    df_clus['lab'][ab]=df['lab'][ab]


/tmp/ipykernel_1375/2785553540.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  cluster_mean=fve(torch.tensor(cluster_data))[1].squeeze(0).squeeze(1)


In [11]:
(len((df_clus["feat"][100])))

10

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.M = 500
        self.L = 128
        self.ATTENTION_BRANCHES = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.M),
            nn.ReLU(),
        )
        self.feature_extractor_part5 = nn.Sequential(
            nn.Linear(1450, self.M),
            nn.ReLU(),
        )
        self.attention = nn.Sequential(
            nn.Linear(self.M, self.L), # matrix V
            nn.Tanh(),
            nn.Linear(self.L, self.ATTENTION_BRANCHES) # matrix w (or vector w if self.ATTENTION_BRANCHES==1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.M*self.ATTENTION_BRANCHES, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.squeeze(0)

        H = self.feature_extractor_part1(x)
        if H.shape==torch.Size([32, 50, 1, 1]):
            H = H.view(-1, 50 * 4 * 4)
            H = self.feature_extractor_part2(H)  # KxM

        else:
            H = H.view(-1, 1450)
            H = self.feature_extractor_part5(H)  # KxM


        A = self.attention(H)  # KxATTENTION_BRANCHES
        A = torch.transpose(A, 1, 0)  # ATTENTION_BRANCHESxK
        A = F.softmax(A, dim=1)  # softmax over K

        Z = torch.mm(A, H)  # ATTENTION_BRANCHESxM

        Y_prob = self.classifier(Z)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().data.item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

class GatedAttention(nn.Module):
    def __init__(self):
        super(GatedAttention, self).__init__()
        self.M = 500
        self.L = 128
        self.ATTENTION_BRANCHES = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.M),
            nn.ReLU(),
        )
        self.feature_extractor_part5 = nn.Sequential(
            nn.Linear(1450, self.M),
            nn.ReLU(),
        )
        self.feature_extractor_part6 = nn.Sequential(
            nn.Linear(50, self.M),
            nn.ReLU(),
        )
        self.attention_V = nn.Sequential(
            nn.Linear(self.M, self.L), # matrix V
            nn.Tanh()
        )

        self.attention_U = nn.Sequential(
            nn.Linear(self.M, self.L), # matrix U
            nn.Sigmoid()
        )

        self.attention_w = nn.Linear(self.L, self.ATTENTION_BRANCHES) # matrix w (or vector w if self.ATTENTION_BRANCHES==1)

        self.classifier = nn.Sequential(
            nn.Linear(self.M*self.ATTENTION_BRANCHES, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.squeeze(0)
        H = self.feature_extractor_part1(x)

        if H.shape==torch.Size([32, 50, 1, 1]):
            H = H.view(-1, 50 * 4 * 4)
            H = self.feature_extractor_part2(H)  # KxM

        else:
#             H = H.view(-1, 1450)
            H = H.view(-1, 50)

#             H = self.feature_extractor_part5(H)  # KxM
            H = self.feature_extractor_part6(H)  # KxM

        A_V = self.attention_V(H)  # KxL
        A_U = self.attention_U(H)  # KxL
        A = self.attention_w(A_V * A_U) # element wise multiplication # KxATTENTION_BRANCHES
        A = torch.transpose(A, 1, 0)  # ATTENTION_BRANCHESxK
        A = F.softmax(A, dim=1)  # softmax over K

        Z = torch.mm(A, H)  # ATTENTION_BRANCHESxM

        Y_prob = self.classifier(Z)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A
import torch
import torch.nn as nn
import torch.nn.functional as F

class GatedAttention1(nn.Module):
    def __init__(self, dropout_rate=0.02, weight_decay=1e-5):
        super(GatedAttention1, self).__init__()
        self.M = 50
        self.L = 128
        self.ATTENTION_BRANCHES = 1
        self.dropout_rate = dropout_rate
        self.weight_decay = weight_decay

        # Feature extractor
        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.M),
            nn.ReLU(),
        )
        
        # Attention mechanism
        self.attention_V = nn.Sequential(
            nn.Linear(self.M, self.L),
            nn.Tanh(),
            nn.Dropout(p=self.dropout_rate)
        )

        self.attention_U = nn.Sequential(
            nn.Linear(self.M, self.L),
            nn.Sigmoid(),
            nn.Dropout(p=self.dropout_rate)
        )

        self.attention_w = nn.Linear(self.L, self.ATTENTION_BRANCHES)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(self.M * self.ATTENTION_BRANCHES, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
#         print(x.shape)
#         x=x.expand(-1, -1, -1, 4)
#         x=x.expand(-1,4,-1)

#         x = x.squeeze(0)
#         print(x.shape)
        H = self.feature_extractor_part1(x)

        if H.shape == torch.Size([32, 50, 1, 1]):
            H = H.view(-1, 50 * 4 * 4)
            H = self.feature_extractor_part2(H)  # KxM

        else:
            H = H.view(-1, 50)

        A_V = self.attention_V(H)  # KxL
        A_U = self.attention_U(H)  # KxL
        A = self.attention_w(A_V * A_U)
        A = torch.transpose(A, 1, 0)  # ATTENTION_BRANCHESxK
        A = F.softmax(A, dim=1)  # softmax over K

        Z = torch.mm(A, H)  # ATTENTION_BRANCHESxM

        Y_prob = self.classifier(Z)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))

        return neg_log_likelihood, A


In [13]:
import numpy as np
# (np.vstack(df_clus["feat"][0]))

In [14]:
for i in range(157):
    df_clus["feat"][i]=np.vstack(df_clus["feat"][i])


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import numpy as np

X = torch.tensor(df_clus['feat'].tolist(), dtype=torch.float32).unsqueeze(1)
y = torch.tensor(df_clus['lab'], dtype=torch.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
device="cuda:6"
model = GatedAttention1()  # or GatedAttention()
model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
print(device)
num_epochs = 8000
abc=0
for epoch in range(num_epochs):

    model.train()
    train_loss = 0.
    train_error = 0.
    a=[]
    p=[]
    for inputs, labels in train_loader:

        inputs=inputs.to(device)
        labels=labels.to(device)
        a.append(labels)
        optimizer.zero_grad()
      #  print(inputs.shape)
     #   print(labels.shape)

        loss, _ = model.calculate_objective(inputs, labels)
#         print(loss)
#         print(inputs.shape)
        zeros = torch.zeros(3, dtype=loss.data[0].dtype)
        if loss.data[0].shape==torch.Size([29]):
            abcd = torch.cat((loss.data[0], zeros))
            train_loss +=abcd

        else:
                train_loss += loss.data[0]


        error, pred = model.calculate_classification_error(inputs, labels)
        p.append(pred)
        train_error += error
        loss.backward()
        optimizer.step()
      #  print(pred)
        correct_predictions = (pred == labels).sum().item()
        total_samples = len(labels)
        accuracy = correct_predictions / total_samples
    #print(p)
    total_elements = len(a)
    correct_count = sum(1 for i in range(total_elements) if a[i].item() == p[i].item())

    accuracy = (correct_count / total_elements) * 100
    a_numpy = np.array([tensor.item() for tensor in a])
    p_numpy = np.array([tensor.item() for tensor in p])


    print(epoch,accuracy)
    if accuracy>80:

        a_flat=a_numpy
        p_flat=p_numpy

        correct_predictions = (a_flat == p_flat)[0].sum().item()
        total_samples = len(a)
        accuracy =correct_predictions / total_samples
        print(correct_predictions,total_samples)
        precision = precision_score(a_flat, p_flat, average='weighted')
        recall = recall_score(a_flat, p_flat, average='weighted')
        f1 = f1_score(a_flat, p_flat, average='weighted')
        auc = roc_auc_score(a_flat, p_flat, average='weighted', multi_class='ovo')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1 Score: {f1:.4f}')
        print(f'AUC: {auc:.4f}')
        abc=abc+1
        if abc>0:
            break

    
model.eval()
test_loss = 0.
test_error = 0.
    
with torch.no_grad():
    a=[]
    p=[]

    for inputs, labels in test_loader:
#             print(labels)
            inputs=inputs.to(device)
            labels=labels.to(device)
            a.append(labels)

            loss, attention_weights = model.calculate_objective(inputs, labels)
            test_loss += loss.data[0]
            error, predicted_label = model.calculate_classification_error(inputs, labels)
            test_error += error
            p.append(predicted_label)
            correct_predictions = (predicted_label == labels).sum().item()
            total_samples = len(labels)
            accuracy = correct_predictions / total_samples
#     print(a)
#     print(p)


    total_elements = len(a)
    correct_count = sum(1 for i in range(total_elements) if a[i].item() == p[i].item())

    accuracy = (correct_count / total_elements) * 100
    print("Test:",accuracy)
    a_numpy = np.array([tensor.item() for tensor in a])
    p_numpy = np.array([tensor.item() for tensor in p])
    test_error /= len(test_loader)
    test_loss /= len(test_loader)
#     print(p)


cuda:6
0 48.226950354609926
1 51.06382978723404
2 58.156028368794324
3 63.829787234042556
4 63.829787234042556
5 65.95744680851064
6 65.95744680851064
7 73.75886524822694
8 72.3404255319149
9 70.2127659574468
10 77.30496453900709
11 78.01418439716312
12 77.30496453900709
13 75.177304964539
14 84.39716312056737
1 141
Precision: 0.8545
Recall: 0.8440
F1 Score: 0.8401
AUC: 0.8275
Test: 81.25
